In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
import logging
import sys

logging.basicConfig(
    stream=sys.stdout,
    # DEBUG: デバッグ用に詳細なログを出力する
    level=logging.DEBUG,
    force=True,
)

In [3]:
from llama_index import SimpleDirectoryReader

# ドキュメントの読み込み
path = "../data"
documents = SimpleDirectoryReader(path).load_data()
print("documents: ", documents)

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 2
documents:  [Document(id_='bb099802-9e7a-4ffd-885f-305f4247850d', embedding=None, metadata={'page_label': '1', 'file_name': 'clever-seals-dive-deep-2001041616-article_only.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='e51c28ce03eaafbf54968aed56e82ca257f6687a229350403366d02bee1e046e', text="This article is available at 5 reading levels at https://newsela.com.How deep is the sea? These clever seals know\nand are happily sharing their secrets\nWith tracking devices attached to their heads, seals can provide us with more information about the ocean floor when they deep dive. Photo:\nClive R. McMahon/IMOS/SIMS Photo: Clive R. McMahon/IMOS/SIMS\nSeals are helping scientists to better understand the deep ocean.\nClive McMahon works on an elephant seal tagging project at the University of Tasmania in\nAustralia. He also works with the Sydney Institute of Marine Sc

In [4]:
# from llama_index import Document

# # ドキュメントを手動で作成
"""何故かエラーとなるためコメントアウト"""
# texts = ["text1", "text2", "text3"]
# documents = [Document(text) for text in texts]
# print("documents: ", documents)
# Document("text1")

In [14]:
from llama_index import VectorStoreIndex

# インデックスの作成
index = VectorStoreIndex.from_documents(documents)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: This article is available at 5 reading levels a...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: This article is available at 5 reading levels a...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: This article is available at 5 reading levels a...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: This article is available at 5 reading levels a...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: This article is available at 5 reading levels a...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: This article is available at 5 reading levels a...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["page_label: 1 file_name: clever-seals-dive-deep-2001041616-article_only.pdf  This article is available at 5 reading levels at https://newsela.com.How deep is the sea? These clever seals know and are happil

In [13]:
from llama_index import VectorStoreIndex

# 空のインデックスを作成
index = VectorStoreIndex([])

# ドキュメントを追加
for doc in documents:
    index.insert(doc)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: This article is available at 5 reading levels a...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["page_label: 1 file_name: clever-seals-dive-deep-2001041616-article_only.pdf  This article is available at 5 reading levels at https://newsela.com.How deep is the sea? These clever seals know and are happily sharing their secrets With tracking devices attached to their heads, seals can provide us with more information about the ocean floor when they deep dive. Photo: Clive R. McMahon/IMOS/SIMS Photo: Clive R. McMahon/IMOS/SIMS Seals are helping scientists to better understand the deep ocean. Clive McMahon works on an elephant seal tagging project at the University of Tasmania in Australia. He also works with the Sydney Institute of Marine Science (SIMS). McMahon said that deep-diving seals can provide scientists with a wealth of informa

### Memo：ノード(Node)とは
「LlamaIndex」内のデータの単位であり、ソースドキュメントの「チャンク」を表す。<br>
（参考： [LlamaIndex v0.7 のしくみとカスタマイズ](https://note.com/npaka/n/n27a36f784fb3) ）

### Memo：LlamaIndexのインデックスの種類
- ベクトルストアインデックス（VectorStoreIndexなど）<br>
各ノードと対応する埋め込み表現（embedding）をベクトルストアに格納するインデックス


- リストインデックス（ListIndex）<br>
ノードをリスト構造で格納するインデックス


- ツリーインデックス（TreeIndex）<br>
一連のノード（リーフノード）で階層ツリーを構築するインデックス


- キーワードテーブルインデックス（KeywordTableIndexなど）<br>
各ノードからキーワードを抽出し、各キーワードから対応するノードへのリンクを構築するインデックス


その他の参考：<br>
[LlamaIndexを完全に理解するチュートリアル その１：処理の概念や流れを理解する基礎編（v0.7.9対応）](https://dev.classmethod.jp/articles/llamaindex-tutorial-001-overview-v0-7-9/#toc-9)

In [12]:
from llama_index import VectorStoreIndex, ServiceContext, LLMPredictor, PromptHelper
from llama_index.embeddings.openai import OpenAIEmbedding
from langchain.chat_models import ChatOpenAI

# llm_predictor: 使用するLLMのカスタマイズ
# デフォルトは"text-davinci-003"
llm_predictor = LLMPredictor(llm=ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
))

# prompt_helper: チャンク分割ルールのカスタマイズ
# デフォルトは以下の通り
prompt_helper = PromptHelper(
    # LLM入力の最大トークン数
    context_window = 4097,
    # LLM出力の最大トークン数
    num_output = 256,
    # チャンクオーバーラップ（チャンクが前後のチャンクと一定量のテキストを重複させる設定）の割合
    chunk_overlap_ratio = 0.1,
)

# embed_model: 埋め込みモデルのカスタマイズ
# デフォルトは"text-embedding-ada-002"（以下の通り）
embed_model = OpenAIEmbedding()

# service_contextの設定
"""他のパラメータ（node_parserなど）も入力しないとエラーとなるためコメントアウト"""
# service_context = ServiceContext(
#     llm_predictor=llm_predictor,
#     prompt_helper=prompt_helper,
#     embed_model=embed_model,
# )

# インデックスの作成
# index = VectorStoreIndex.from_documents(documents, service_context=service_context)

'他のパラメータ（node_parserなど）も入力しないとエラーとなるためコメントアウト'

In [15]:
# クエリエンジンの作成
query_engine = index.as_query_engine()

In [17]:
response = query_engine.query("What is Hamilton? Please explain it to me shortly.")

# response.response: 回答
print('---------------------------------')
print(response.response)

# response.source_nodes: 回答を生成する際に利用したソース
print('---------------------------------')
print(response.source_nodes)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["What is Hamilton? Please explain it to me shortly."], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=18 request_id=d62deba9ea6f04a6924dafcd1e53dc72 response_code=200
DEBUG:llama_index.indices.utils:> Top 2 nodes:
> [Node 7b9a4753-3b45-48bd-aef9-db2e62464740] [Similarity score:             0.797944] This article is available at 5 reading levels at https://newsela.com.Edelman says the simulator w...
> [Node 5c43e23e-26a7-4d7d-9284-205c1a14ef8b] [Similarity score:             0.796045] This article is available at 5 reading levels at https://newsela.com.If you’ve ever wanted to be ...
DEBUG:openai:message='Reques